In [1]:
import numpy as np
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow

In [2]:
import numba
from numba import vectorize,njit,jit

In [3]:
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [10]:
import cv2
from skimage.color import rgb2hsv,hsv2rgb
import numpy as np

@jit
def  HSVStretching(sceneRadiance):
    sceneRadiance = np.uint8(sceneRadiance)
    height = len(sceneRadiance)
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    h, s, v = cv2.split(img_hsv)
    img_s_stretching = global_stretching(s, height, width)
    img_v_stretching = global_stretching(v, height, width)

    labArray = np.zeros((height, width, 3), 'float64')
    labArray[:, :, 0] = h
    labArray[:, :, 1] = img_s_stretching
    labArray[:, :, 2] = img_v_stretching
    img_rgb = hsv2rgb(labArray) * 255

    # img_rgb = np.clip(img_rgb, 0, 255)

    return img_rgb

In [11]:
import numpy as np

@jit
def sceneRadianceRGB(sceneRadiance):

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance



In [12]:
import numpy as np

@jit
def global_stretching(img_L,height, width):
    I_min = np.min(img_L)
    I_max = np.max(img_L)
    I_mean = np.mean(img_L)


    # print('I_min',I_min)
    # print('I_max',I_max)
    # print('I_max',I_mean)

    array_Global_histogram_stretching_L = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            p_out = (img_L[i][j] - I_min) * ((1) / (I_max - I_min))
            array_Global_histogram_stretching_L[i][j] = p_out

    return array_Global_histogram_stretching_L



In [17]:
import numpy as np
import concurrent.futures
import copy
import time

@jit
def histogram_general(r_array,height, width,input):
    length = height * width

    R_rray=np.zeros(r_array.shape)
    R_rray = np.copy(r_array)
    R_rray=R_rray.flatten()
    R_rray.sort()
    
    I_min = int(R_rray[int(length / 500)])
    I_max = int(R_rray[-int(length / 500)])
    array_Global_histogram_stretching = np.zeros((height, width))
    if input == 0:
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_min
            elif (r_array[i][j] > I_max):
                p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 255
            else:
                p_out = int((r_array[i][j] - I_min) * ((255 - I_min) / (I_max - I_min)))+ I_min
                array_Global_histogram_stretching[i][j] = p_out

    elif input == 1:
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                  p_out = r_array[i][j]
                  array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                  p_out = r_array[i][j]
                  array_Global_histogram_stretching[i][j] = 255
            else:
                  p_out = int((r_array[i][j] - I_min) * ((255) / (I_max - I_min)))
                  array_Global_histogram_stretching[i][j] = p_out
    elif input == 2 :
      for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_max
            else:
                p_out = int((r_array[i][j] - I_min) * ((I_max) / (I_max - I_min)))
                array_Global_histogram_stretching[i][j] = p_out

    return (array_Global_histogram_stretching)

def stretching(img):
    height = len(img)
    width = len(img[0])
    with concurrent.futures.ThreadPoolExecutor() as executor:
      input=[img[:, :, 2],img[:, :, 1],img[:, :, 0]]
      inp=[0,1,2]
      h=[height,height,height]
      w=[width,width,width]
    
      results=executor.map(histogram_general,input,h,w,inp)
      i=0
      for result in results:
        input[i]=result
        i=i+1
      
    return img




In [18]:
import numpy as np
@jit
def cal_equalisation(img,ratio):
    Array = img * ratio
    Array = np.clip(Array, 0, 255)
    return Array

@jit
def RGB_equalisation(img):
    i =0
    img = np.float32(img)
    avg_RGB = []
    for i in range(3):
        avg = np.mean(img[:,:,i])
        avg_RGB.append(avg)
    # print('avg_RGB',avg_RGB)
    a_r = avg_RGB[0]/avg_RGB[2]
    a_g =  avg_RGB[0]/avg_RGB[1]
    ratio = [0,a_g,a_r]
    for i in range(1,3):
        img[:,:,i] = cal_equalisation(img[:,:,i],ratio[i])
    return img






In [19]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime
import timeit

folder = "/content/gdrive/My Drive"
pathinp = folder + "/auv_files/test_images"
pathout = folder + "/auv_files/out_test_images"
filesinp = os.listdir(pathinp)
filesoup = os.listdir(pathout)
length = len(filesinp)
print(filesinp)

['rod4.jpg', 'rod3.jpg', 'rod2.jpg', 'rod1.jpg']


In [20]:
starti = timeit.default_timer()
def process_image(imginp):
  imgdir = pathinp + "/" + imginp
  img = cv2.imread(imgdir)

  # start = timeit.default_timer()
  sceneRadiance = RGB_equalisation(img)
  # stop = timeit.default_timer()
  # print('RGB_equalisation: ', stop - start)

  # start = timeit.default_timer()
  sceneRadiance = stretching(sceneRadiance)
  # stop = timeit.default_timer()
  # print('stretching: ', stop - start)

  # start = timeit.default_timer()
  sceneRadiance = HSVStretching(sceneRadiance)
  # stop = timeit.default_timer()
  # print('HSVStretching: ', stop - start)

  # start = timeit.default_timer()
  sceneRadiance = sceneRadianceRGB(sceneRadiance)
  # stop = timeit.default_timer()
  # print('sceneRadianceRGB: ', stop - start)
  return sceneRadiance

i=0
with concurrent.futures.ProcessPoolExecutor() as executor:
  results=executor.map(process_image,filesinp)
  for result in results:
    cv2.imwrite('/content/gdrive/My Drive/auv_files/out_test_images/'+ str(i) +'.jpg',result)
    i=i+1

stopi = timeit.default_timer()
print('Time: ', stopi - starti)

<ipython-input-18-a165dec9414d>:8: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "RGB_equalisation" failed type inference due to: No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(float32, Tuple(slice<a:b>, slice<a:b>, int64))
 
There are 22 candidate implementations:
   - Of which 22 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(float32, Tuple(slice<a:b>, slice<a:b>, int64))':
    No match.

During: typing of intrinsic-call at <ipython-input-18-a165dec9414d> (14)

File "<ipython-input-18-a165dec9414d>", line 14:
def RGB_equalisation(img):
    <source elided>
    for i in range(3):
        avg = np.mean(img[:,:,i])
        ^

  @jit
<ipython-input-18-a165dec9414d>:8: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "RGB_equalisation" failed type inference due to: No 

Time:  15.461293651000005


In [21]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime


np.seterr(over='ignore')
if __name__ == '__main__':
    pass

folder = "/content/gdrive/My Drive"

# path = sett
pathinp = folder + "/auv_files/test_images"
pathout = folder + "/auv_files/out_test_images"
filesinp = os.listdir(pathinp)
filesoup = os.listdir(pathout)
length = len(filesinp)
for i in range(length):
    imginp = filesinp[i]
    imgdir = pathinp + "/" + imginp
    # print(filesinp[i])
    print('********    file   ********',imgdir)
    # img = cv2.imread('InputImages/' + file)
    import timeit
    start = timeit.default_timer()
    img = cv2.imread(imgdir)
    print(len(img))
    #img = cv2.resize(img,(256,256))
    #cv2_imshow(img)
    sceneRadiance = RGB_equalisation(img)
    sceneRadiance = stretching(sceneRadiance)
    # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
    sceneRadiance = HSVStretching(sceneRadiance)
    sceneRadiance = sceneRadianceRGB(sceneRadiance)
    cv2.imwrite('/content/gdrive/My Drive/auv_files/out_test_images/'+ "oup" + imginp,sceneRadiance)
    #cv2_imshow(sceneRadiance)
    stop = timeit.default_timer()
    print('Time: ', stop - start)  



********    file   ******** /content/gdrive/My Drive/auv_files/test_images/rod4.jpg
1080
0.023601055145263672
Time:  68.50577573100003
********    file   ******** /content/gdrive/My Drive/auv_files/test_images/rod3.jpg
1080
0.015539884567260742


KeyboardInterrupt: ignored

In [ ]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime
import timeit

folder = "/content/gdrive/My Drive"
pathinp = folder + "/auv_files/test_images"
pathout = folder + "/auv_files/out_test_images"
filesinp = os.listdir(pathinp)
filesoup = os.listdir(pathout)
length = len(filesinp)
print(filesinp)

In [ ]:
starti = timeit.default_timer()
def process_image(imginp):
  imgdir = pathinp + "/" + imginp
  img = cv2.imread(imgdir)

  start = timeit.default_timer()
  sceneRadiance = RGB_equalisation(img)
  stop = timeit.default_timer()
  print('RGB_equalisation: ', stop - start)

  start = timeit.default_timer()
  sceneRadiance = stretching(sceneRadiance)
  stop = timeit.default_timer()
  print('stretching: ', stop - start)

  start = timeit.default_timer()
  sceneRadiance = HSVStretching(sceneRadiance)
  stop = timeit.default_timer()
  print('HSVStretching: ', stop - start)

  start = timeit.default_timer()
  sceneRadiance = sceneRadianceRGB(sceneRadiance)
  stop = timeit.default_timer()
  print('sceneRadianceRGB: ', stop - start)
  return sceneRadiance

i=0
with concurrent.futures.ProcessPoolExecutor() as executor:
  results=executor.map(process_image,filesinp)
  for result in results:
    cv2.imwrite('/content/gdrive/My Drive/auv_files/out_test_images/'+ str(i) +'.jpg',result)
    i=i+1

stopi = timeit.default_timer()
print('Time: ', stopi - starti) 


In [ ]:
# cap = cv2.VideoCapture(video1)
# ret,frame=cap.read()
# count=0
# out = cv2.VideoWriter('/content/gdrive/My Drive/auv_files/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame.shape[1], frame.shape[0]))
# while cap.isOpened():
#     import timeit
#     starti = timeit.default_timer()
#     ret, frame = cap.read()

#     # check for successfulness of cap.read()
#     if not ret: break
#     start = timeit.default_timer()
#     sceneRadiance = RGB_equalisation(frame)
#     stop = timeit.default_timer()
#     print('RGB_equalisation: ', stop - start ,count)

#     start = timeit.default_timer()
#     sceneRadiance = stretching(sceneRadiance)
#     stop = timeit.default_timer()
#     print('stretching ', stop - start ,count)
#     # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
#     start = timeit.default_timer()
#     sceneRadiance = HSVStretching(sceneRadiance)
#     stop = timeit.default_timer()
#     print('HSVStretching: ', stop - start ,count)

#     start = timeit.default_timer()
#     sceneRadiance = sceneRadianceRGB(sceneRadiance)
#     stop = timeit.default_timer()
#     print('sceneRadianceRGB: ', stop - start ,count)
#     # Save the video
#     out.write(sceneRadiance)
#     stopi = timeit.default_timer()
#     print('Time per image ', stopi - starti ,count)
    # cv2_imshow(image) # Note cv2_imshow, not cv2.imshow

    # cv2.waitKey()

# cv2.destroyAllWindows()
# cap.release()
# out.release()

In [ ]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime

np.seterr(over='ignore')
if __name__ == '__main__':
    pass


cap = cv2.VideoCapture(video1)
ret,frame=cap.read()
count=0
frames=[]
out = cv2.VideoWriter('/content/gdrive/My Drive/auv_files/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1], frame.shape[0]))
while cap.isOpened():
    # import timeit
    # starti = timeit.default_timer()
    ret, frame = cap.read()
    if ret:
      frames.append(frame)
    if not ret: break

cap.release()
print(len(frames))
# out.release()

In [ ]:
# check for successfulness of cap.read()
starti = timeit.default_timer()
def process_image(img_frame):
    start = timeit.default_timer()
    sceneRadiance = RGB_equalisation(img_frame)
    stop = timeit.default_timer()
    print('RGB_equalisation: ', stop - start ,count)

    start = timeit.default_timer()
    sceneRadiance = stretching(sceneRadiance)
    stop = timeit.default_timer()
    print('stretching ', stop - start ,count)
    # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
    start = timeit.default_timer()
    sceneRadiance = HSVStretching(sceneRadiance)
    stop = timeit.default_timer()
    print('HSVStretching: ', stop - start ,count)

    start = timeit.default_timer()
    sceneRadiance = sceneRadianceRGB(sceneRadiance)
    stop = timeit.default_timer()
    print('sceneRadianceRGB: ', stop - start ,count)
    # Save the video
    # out.write(sceneRadiance)
    
    return sceneRadiance 


with concurrent.futures.ProcessPoolExecutor() as executor:
  results=executor.map(process_image,frames)
  for result in results:
    out.write(result)
out.release()
stopi = timeit.default_timer()
print('Time per image ', stopi - starti)

In [ ]:
import numba